In [ ]:
import ipywidgets as widgets
import redis
import json

In [ ]:
import subprocess

positionv=[]
def addposition(x,y):
        positionv.append((x,y))
    
def isDetect(line):
    return line.startswith("detection=")
def toPosition(line):
      plotline=line[10:]
      plotv=plotline.split(",")
      return plotv
def apriltag(imagename):
#    imagename='/Users/aadebuger/GEXT/cloud2019/machinevision/data/tz_belt_output20190417_5/40.jpg'
    
#    lines=subprocess.getoutput('/Users/aadebuger/GEXT/github2019/apriltags/xcodebuild/Debug/apriltag_demo {0}'.format(imagename))
    lines=subprocess.getoutput('/home/aadebugergf/Ai/cloud2019/apriltags/build/apriltag_demo {0}'.format(imagename))
    linev=lines.split("\n")
    detectionv=filter(lambda item:isDetect(item),linev)
    positionv=map(lambda item:toPosition(item),detectionv)
    return list(positionv)
import numpy as np 
def getBox(plotv):
            arr2=np.asarray(plotv[1:],float)
            rect=(arr2[6],arr2[7],arr2[4]-arr2[0],arr2[1]-arr2[5])
            return (rect[0]+rect[2]/2,rect[1]+rect[3]/2)

def computeposition(x,y):
    yuandian=(853.5124299065,706.912679109)
    kx = -0.8937383178 
    ky = -0.8729680915
    scarax = yuandian[0] + kx*x 
    scaray = yuandian[1] + ky*y
    return (scarax,scaray)
r = redis.Redis(host='localhost', port=6379, db=0)
import base64
#import cStringIO
import sys
import tempfile
import time
MODEL_BASE = '/Users/aadebuger/GEXT/github2019/models/research'
MODEL_BASE = '/home/aadebugergf/Ai/github2019/models/research'


sys.path.append(MODEL_BASE)
sys.path.append(MODEL_BASE + '/object_detection')
sys.path.append(MODEL_BASE + '/slim')


import numpy as np
from PIL import Image
from PIL import ImageDraw
import tensorflow as tf
from utils import label_map_util




PATH_TO_CKPT = '/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = MODEL_BASE + '/object_detection/data/mscoco_label_map.pbtxt'

content_types = {'jpg': 'image/jpeg',
                 'jpeg': 'image/jpeg',
                 'png': 'image/png'}
extensions = sorted(content_types.keys())


def is_image():
  def _is_image(form, field):
    if not field.data:
      raise ValidationError()
    elif field.data.filename.split('.')[-1].lower() not in extensions:
      raise ValidationError()

  return _is_image





class ObjectDetector(object):

  def __init__(self):
    self.detection_graph = self._build_graph()
    self.sess = tf.Session(graph=self.detection_graph)

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    self.category_index = label_map_util.create_category_index(categories)

  def _build_graph(self):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    return detection_graph

  def _load_image_into_numpy_array(self, image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

  def detect(self, image):
    image_np = self._load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = self.detection_graph
    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    boxes = graph.get_tensor_by_name('detection_boxes:0')
    scores = graph.get_tensor_by_name('detection_scores:0')
    classes = graph.get_tensor_by_name('detection_classes:0')
    num_detections = graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = self.sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    boxes, scores, classes, num_detections = map(
        np.squeeze, [boxes, scores, classes, num_detections])

    return boxes, scores, classes.astype(int), num_detections
 

def draw_bounding_box_on_image(image, box, color='red', thickness=4):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  ymin, xmin, ymax, xmax = box
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  if (right-left)<100:
        print("too lower")
        return None
  print("rect,width="+str(right-left)+"height="+str(bottom-top))
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  print('(left, right, top, bottom)',(left, right, top, bottom))
  centerx=left+(right-left)/2
  centery=top+ (bottom-top)/2
  print("center=",centerx,"y=",centery)
  scaraxy=computeposition(centerx*4/3,centery*4/3)
  print("scara xy",scaraxy)
  return scaraxy
    
from io import BytesIO
def encode_image(image):
  image_buffer = BytesIO()
  image.save(image_buffer, format='PNG')
  imagevalue = image_buffer.getvalue()
  base64str=base64.b64encode(imagevalue).decode()

  imgstr = 'data:image/png;base64,{:s}'.format(
     base64str )
  return imgstr


def detect_objects(image_path):
  start = time.time()
  
  image = Image.open(image_path).convert('RGB')
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
  print(num_detections)
  new_images = {}
  for i in range(int(num_detections)):
    
    if scores[i] < 0.8: continue
    print("scres[i",scores[i])
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)

  result = {}
#  result['original'] = encode_image(image.copy())
#  print(new_images)
  count = 0 
  for cls, new_image in new_images.items():
    category = client.category_index[cls]['name']
    print(category)
    result[category] = encode_image(new_image)
    new_image.save("{0}.jpg".format(count))
    count=count+1
  end = time.time()
  print("execute2 time",(end-start)) 
  return result

def detect_objectsbyframe(frame):
  start = time.time()
  
#  image = Image.open(image_path).convert('RGB')
  height, width, channels = frame.shape
  image= Image.frombytes("RGB", (width,height), frame.tostring())
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
#  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
#  print(num_detections)
  new_images = {}
  
  for i in range(int(num_detections)):

    if scores[i] < 0.8: continue
    print("scres[i",scores[i])
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    box=draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)
    if box is not None:
        print("scarxbox",box)
        return (box,new_images[cls])
    return (None,None)
#  result['original'] = encode_image(image.copy())
#  print(new_images)
  return (None,None)


PATH_TO_CKPT='/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'
PATH_TO_CKPT='/home/aadebugergf/Ai/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'


PATH_TO_CKPT='/home/aadebugergf/Ai/models/fastrcnnstbmodelrtx2/frozen_inference_graph.pb'
client = ObjectDetector()

In [ ]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052800.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        start1 = time.time()
        boxframe=frame[0:480,0:200]
        box,newframe=detect_objectsbyframe(boxframe)
        end1 = time.time()
        print("execute1 time",(end1-start1)) 
        if box is not None:
            print("box",box)

            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

In [6]:
print(boxv)

[(771.3077680854631, 358.1324317794112), (770.5208580622261, 358.3664551887975), (770.5643591240421, 356.8387968273442), (768.8029795031067, 356.907717595562), (767.1733613275752, 356.5309199257961), (765.4248562310332, 356.05098461792346), (765.8966781444855, 355.9056628241653), (765.055949540877, 355.5519428097113), (762.101353799913, 355.195487943248), (761.0603296019324, 355.17663120108296), (760.1786771853087, 355.56043458764), (754.4793604808727, 355.0764407094337), (749.2850811957223, 355.43135956312466), (746.8554355570059, 356.02170047197836), (746.0996805190646, 355.78833892175413), (743.9956248261926, 354.277801483154), (739.2004675168487, 355.4308350709585), (736.9314465923496, 356.1794102711854), (736.3148864288667, 355.09617160520907), (734.2777284012267, 355.197660839365), (731.1853004568321, 355.2985131901763), (728.6137175548964, 355.3401603657528), (726.1538523688533, 356.03392613461386), (724.0621065282522, 356.17796167377406), (722.773322315035, 355.4234422289971), 

In [8]:
import pandas as pd

In [9]:
boxf = pd.DataFrame(boxv)

In [10]:
print(boxf)

              0           1
0    771.307768  358.132432
1    770.520858  358.366455
2    770.564359  356.838797
3    768.802980  356.907718
4    767.173361  356.530920
5    765.424856  356.050985
6    765.896678  355.905663
7    765.055950  355.551943
8    762.101354  355.195488
9    761.060330  355.176631
10   760.178677  355.560435
11   754.479360  355.076441
12   749.285081  355.431360
13   746.855436  356.021700
14   746.099681  355.788339
15   743.995625  354.277801
16   739.200468  355.430835
17   736.931447  356.179410
18   736.314886  355.096172
19   734.277728  355.197661
20   731.185300  355.298513
21   728.613718  355.340160
22   726.153852  356.033926
23   724.062107  356.177962
24   722.773322  355.423442
25   720.017652  355.784568
26   716.329252  354.817978
27   713.980376  355.256054
28   711.650181  355.280968
29   710.016356  355.037428
..          ...         ...
77   588.617603  359.943553
78   586.518762  357.131226
79   583.787581  358.284522
80   578.975271  357

In [13]:
print(boxf.diff(1))

            0         1
0         NaN       NaN
1   -0.786910  0.234023
2    0.043501 -1.527658
3   -1.761380  0.068921
4   -1.629618 -0.376798
5   -1.748505 -0.479935
6    0.471822 -0.145322
7   -0.840729 -0.353720
8   -2.954596 -0.356455
9   -1.041024 -0.018857
10  -0.881652  0.383803
11  -5.699317 -0.483994
12  -5.194279  0.354919
13  -2.429646  0.590341
14  -0.755755 -0.233362
15  -2.104056 -1.510537
16  -4.795157  1.153034
17  -2.269021  0.748575
18  -0.616560 -1.083239
19  -2.037158  0.101489
20  -3.092428  0.100852
21  -2.571583  0.041647
22  -2.459865  0.693766
23  -2.091746  0.144036
24  -1.288784 -0.754519
25  -2.755670  0.361125
26  -3.688400 -0.966589
27  -2.348876  0.438076
28  -2.330195  0.024913
29  -1.633824 -0.243539
..        ...       ...
77  -0.485431  1.368425
78  -2.098842 -2.812327
79  -2.731181  1.153296
80  -4.812311 -0.645712
81  -1.819327  2.112030
82  -1.143996 -0.797478
83  -0.925619 -0.057819
84  -6.552830 -1.936275
85  -2.083176  1.553184
86  -2.751236 -0

In [ ]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019053102.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

In [15]:
boxf = pd.DataFrame(boxv)
print(boxf)

             0           1
0   792.594653  381.565968
1   791.540513  380.843654
2   790.198363  381.358581
3   786.927597  382.031279
4   785.228818  382.491384
5   780.728131  382.974304
6   778.854114  382.227127
7   776.855449  382.485078
8   774.826226  383.037156
9   773.443210  383.140668
10  771.029925  382.853321
11  767.109316  383.009432
12  766.196827  384.038773
13  765.930483  383.054926
14  762.422880  383.608765
15  759.806640  382.535092
16  758.700390  381.440676
17  756.277891  383.624712
18  753.740776  382.298471
19  749.615211  383.011331
20  746.470613  383.509049
21  743.178915  384.363471
22  743.454473  383.329098
23  740.328531  384.162578
24  736.223765  384.012536
25  734.903556  385.329648
26  733.038646  383.552969
27  730.760967  384.561192
28  730.282456  382.706613
29  724.220517  381.894449
30  723.539459  384.151864
31  720.185330  385.026679
32  719.422980  383.808446
33  717.690506  385.468389
34  713.627752  383.850531
35  712.769721  383.137071
3

In [4]:
cap.release()

In [3]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019062100.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.96097124
boxes [0.6617025  0.1929848  0.94098765 0.39714018]
too lower
scres[i 0.9720372
boxes [0.6663183  0.19755977 0.9424755  0.38162938]
too lower
scres[i 0.91926014
boxes [0.6671885  0.19786282 0.9445441  0.3838324 ]
too lower
scres[i 0.9373555
boxes [0.6636067  0.19829275 0.9431057  0.38315588]
too lower
scres[i 0.94605297
boxes [0.66799724 0.18871167 0.9387339  0.41564062]
rect,width=108.92589569091797height=97.46518850326538
(left, right, top, bottom) (90.58160305023193, 199.5074987411499, 240.47900676727295, 337.94419527053833)
center= 145.04455089569092 y= 289.21160101890564
scara xy (680.6699326750715, 370.28267993415534)
scarxbox (680.6699326750715, 370.28267993415534)
box (680.6699326750715, 370.28267993415534)
scres[i 0.9788916
boxes [0.67064476 0.18540388 0.9461826  0.42306364]
rect,width=114.07668113708496height=99.19362545013428
(left, right, top, bottom) (88.99386405944824, 203.0705451965332, 241.43211364746094, 340.6257390975952)
center= 146.03220462799072 

scres[i 0.93509287
boxes [0.65839    0.23266509 0.9766152  0.46178383]
rect,width=109.9769926071167height=114.56107378005981
(left, right, top, bottom) (111.67924404144287, 221.65623664855957, 237.0203948020935, 351.5814685821533)
center= 166.66774034500122 y= 294.3009316921234
scara xy (654.9026353898752, 364.3589155544069)
scarxbox (654.9026353898752, 364.3589155544069)
box (654.9026353898752, 364.3589155544069)
scres[i 0.9492437
boxes [0.66136646 0.24339183 0.9453392  0.43874544]
too lower
scres[i 0.9146065
boxes [0.66078377 0.23989773 0.9673323  0.46908563]
rect,width=110.01019477844238height=110.35747289657593
(left, right, top, bottom) (115.15090942382812, 225.1611042022705, 237.8821563720703, 348.23962926864624)
center= 170.15600681304932 y= 293.0608928203583
scara xy (650.7458388496199, 365.8022680441011)
scarxbox (650.7458388496199, 365.8022680441011)
box (650.7458388496199, 365.8022680441011)
scres[i 0.8983635
boxes [0.6623841  0.23804733 0.97151756 0.4623488 ]
rect,width=107

scres[i 0.9678956
boxes [0.6706622  0.27070117 0.94430137 0.5106886 ]
rect,width=115.19396781921387height=98.51009130477905
(left, right, top, bottom) (129.93656158447266, 245.13052940368652, 241.43840074539185, 339.9484920501709)
center= 187.5335454940796 y= 290.6934463977814
scara xy (630.037875931902, 368.55787495776167)
scarxbox (630.037875931902, 368.55787495776167)
box (630.037875931902, 368.55787495776167)
scres[i 0.9399184
boxes [0.66171837 0.280938   0.968948   0.50937134]
rect,width=109.64800357818604height=110.60266971588135
(left, right, top, bottom) (134.85023975372314, 244.49824333190918, 238.21861267089844, 348.8212823867798)
center= 189.67424154281616 y= 293.5199475288391
scara xy (627.48691315121, 365.2679478937589)
scarxbox (627.48691315121, 365.2679478937589)
box (627.48691315121, 365.2679478937589)
scres[i 0.904734
boxes [0.6748807  0.27343333 0.9559316  0.4953239 ]
rect,width=106.50747299194336height=101.17833137512207
(left, right, top, bottom) (131.24799728393555

scres[i 0.9709146
boxes [0.670691   0.3132504  0.94717747 0.5521815 ]
rect,width=114.68692302703857height=99.5351243019104
(left, right, top, bottom) (150.3601884841919, 265.04711151123047, 241.4487648010254, 340.9838891029358)
center= 207.70364999771118 y= 291.2163269519806
scara xy (606.0021489066675, 367.94926423845266)
scarxbox (606.0021489066675, 367.94926423845266)
box (606.0021489066675, 367.94926423845266)
scres[i 0.9467945
boxes [0.6730399  0.31230846 0.94463694 0.55562615]
rect,width=116.79249286651611height=97.77493000030518
(left, right, top, bottom) (149.90806102752686, 266.70055389404297, 242.29436874389648, 340.06929874420166)
center= 208.3043074607849 y= 291.1818337440491
scara xy (605.2863747858388, 367.9894128649829)
scarxbox (605.2863747858388, 367.9894128649829)
box (605.2863747858388, 367.9894128649829)
scres[i 0.976953
boxes [0.6719665  0.3165595  0.94638246 0.55604506]
rect,width=114.95306968688965height=98.78974914550781
(left, right, top, bottom) (151.948556900

scres[i 0.9051925
boxes [0.6588669  0.35348713 0.9413367  0.5915345 ]
rect,width=114.26273345947266height=101.68913125991821
(left, right, top, bottom) (169.67382431030273, 283.9365577697754, 237.19207763671875, 338.88120889663696)
center= 226.80519104003906 y= 288.03664326667786
scara xy (583.2397766952571, 371.6502807695625)
scarxbox (583.2397766952571, 371.6502807695625)
box (583.2397766952571, 371.6502807695625)
scres[i 0.8695811
boxes [0.6606563  0.36624146 0.93934774 0.5757981 ]
rect,width=100.58718681335449height=100.32892942428589
(left, right, top, bottom) (175.7958984375, 276.3830852508545, 237.83625841140747, 338.16518783569336)
center= 226.08949184417725 y= 288.0007231235504
scara xy (584.0926404224042, 371.6920902879524)
scarxbox (584.0926404224042, 371.6920902879524)
box (584.0926404224042, 371.6920902879524)
scres[i 0.9631075
boxes [0.6720731  0.3582216  0.94050455 0.59834176]
rect,width=115.25768280029297height=96.63531303405762
(left, right, top, bottom) (171.946363449

scres[i 0.9003177
boxes [0.66264755 0.39696234 0.9408018  0.62677383]
rect,width=110.30951499938965height=100.13553142547607
(left, right, top, bottom) (190.54192543029785, 300.8514404296875, 238.55311632156372, 338.6886477470398)
center= 245.69668292999268 y= 288.62088203430176
scara xy (560.7277097853112, 370.97025170029195)
scarxbox (560.7277097853112, 370.97025170029195)
box (560.7277097853112, 370.97025170029195)
scres[i 0.9406736
boxes [0.6716102 0.3991389 0.9424696 0.6341936]
rect,width=112.82625675201416height=97.50939130783081
(left, right, top, bottom) (191.58667087554932, 304.4129276275635, 241.77966356277466, 339.28905487060547)
center= 247.9997992515564 y= 290.53435921669006
scara xy (557.9831987094017, 368.7430456682409)
scarxbox (557.9831987094017, 368.7430456682409)
box (557.9831987094017, 368.7430456682409)
scres[i 0.9711639
boxes [0.6718899  0.3996569  0.94299173 0.6379537 ]
rect,width=114.38246726989746height=97.59665966033936
(left, right, top, bottom) (191.83530807

In [5]:
import pandas as pd
boxf = pd.DataFrame(boxv)
print(boxf)

              0           1
0    680.669933  370.282680
1    679.492994  368.167391
2    680.268666  368.297415
3    680.343420  367.792129
4    679.115290  367.848562
5    679.964885  364.842897
6    672.561509  368.097283
7    672.329704  369.193135
8    672.982192  368.058558
9    671.024404  367.962364
10   671.793347  366.159560
11   670.952654  368.999148
12   665.203977  365.412845
13   663.390467  368.351737
14   662.157164  368.564419
15   659.503707  368.513730
16   660.382482  370.608402
17   657.066463  367.229823
18   658.305056  369.794353
19   654.902635  364.358916
20   650.745839  365.802268
21   653.201758  364.590117
22   649.409377  368.034994
23   647.645853  367.289103
24   646.782079  367.963888
25   648.224277  366.244989
26   646.508454  370.363964
27   644.538605  364.472118
28   644.591339  364.985571
29   646.050179  365.358685
..          ...         ...
76   585.799408  370.726787
77   585.192698  370.419110
78   584.148656  370.905577
79   583.239777  371

In [6]:
boxf.describe()

,0,1
count,106.000000,106.000000
mean,615.511530,368.272652
std,37.329393,1.974822
min,553.879844,364.358916
25%,584.106644,367.244643
50%,613.390732,368.255330
75%,646.393885,369.799904
max,680.669933,372.073384


In [9]:
boxf[1].std()

1.974821532011553

In [10]:
boxf[1].mean()

368.27265230073374

In [11]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019062101.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.9933931
boxes [0.68625444 0.00584904 0.94999176 0.15356201]
too lower
scres[i 0.99801624
boxes [0.6877778  0.00504655 0.94977915 0.1557816 ]
too lower
scres[i 0.99752396
boxes [0.68514305 0.00731955 0.9423216  0.15918833]
too lower
scres[i 0.99956805
boxes [0.68516946 0.00305387 0.95510906 0.15778412]
too lower
scres[i 0.99934584
boxes [0.6864255  0.0050125  0.9501876  0.16909644]
too lower
scres[i 0.9989672
boxes [0.68697333 0.0075194  0.9532221  0.16524343]
too lower
scres[i 0.99596125
boxes [0.6842782  0.00795793 0.9483182  0.17235683]
too lower
scres[i 0.99859184
boxes [0.6926397  0.00639924 0.9551763  0.17171787]
too lower
scres[i 0.9989857
boxes [0.6867976  0.00696325 0.9504897  0.1791939 ]
too lower
scres[i 0.9993813
boxes [0.69369656 0.00745908 0.96066165 0.17562404]
too lower
scres[i 0.99966717
boxes [0.69054025 0.00873203 0.95823807 0.18145534]
too lower
scres[i 0.999065
boxes [0.69110847 0.00858688 0.9569994  0.18052219]
too lower
scres[i 0.9992373
boxes [0.6924239

scres[i 0.9730583
boxes [0.6839259  0.03892595 0.9711123  0.2595391 ]
rect,width=105.89430928230286height=103.38709831237793
(left, right, top, bottom) (18.684457540512085, 124.57876682281494, 246.21333360671997, 349.6004319190979)
center= 71.63161218166351 y= 297.90688276290894
scara xy (768.1525411431107, 360.16174192199884)
scarxbox (768.1525411431107, 360.16174192199884)
box (768.1525411431107, 360.16174192199884)
scres[i 0.9881406
boxes [0.6833117  0.04073728 0.9811093  0.25906214]
rect,width=104.79593575000763height=107.20714330673218
(left, right, top, bottom) (19.553892016410828, 124.34982776641846, 245.99221229553223, 353.1993556022644)
center= 71.95185989141464 y= 299.5957839488983
scara xy (767.7709176105877, 358.1959327952452)
scarxbox (767.7709176105877, 358.1959327952452)
box (767.7709176105877, 358.1959327952452)
scres[i 0.9914483
boxes [0.6875412  0.04297606 0.97638154 0.26526755]
rect,width=106.69991791248322height=103.98252725601196
(left, right, top, bottom) (20.6285

scres[i 0.9952199
boxes [0.6894171  0.09204705 0.9896932  0.31189227]
rect,width=105.52570581436157height=108.09939622879028
(left, right, top, bottom) (44.182584285736084, 149.70829010009766, 248.19016456604004, 356.2895607948303)
center= 96.94543719291687 y= 302.2398626804352
scara xy (737.9872939662558, 355.1183376431846)
scarxbox (737.9872939662558, 355.1183376431846)
box (737.9872939662558, 355.1183376431846)
scres[i 0.9728986
boxes [0.6853889  0.09029063 0.97516906 0.31360835]
rect,width=107.19250559806824height=104.32085037231445
(left, right, top, bottom) (43.33950161933899, 150.53200721740723, 246.74001216888428, 351.06086254119873)
center= 96.93575441837311 y= 298.9004373550415
scara xy (737.998832455099, 359.0052866472055)
scarxbox (737.998832455099, 359.0052866472055)
box (737.998832455099, 359.0052866472055)
scres[i 0.97147465
boxes [0.6896289  0.10065904 0.9852676  0.32118285]
rect,width=105.85142612457275height=106.42992496490479
(left, right, top, bottom) (48.3163404464

scres[i 0.9782446
boxes [0.68215966 0.13835952 0.96088016 0.38123894]
rect,width=116.58212184906006height=100.33937931060791
(left, right, top, bottom) (66.41256809234619, 182.99468994140625, 245.57747840881348, 345.9168577194214)
center= 124.70362901687622 y= 295.74716806411743
scara xy (704.9092144117024, 362.6755579470502)
scarxbox (704.9092144117024, 362.6755579470502)
box (704.9092144117024, 362.6755579470502)
scres[i 0.9894475
boxes [0.680055   0.14691566 0.9808885  0.37298167]
rect,width=108.51168394088745height=108.30004692077637
(left, right, top, bottom) (70.5195164680481, 179.03120040893555, 244.81980800628662, 353.119854927063)
center= 124.77535843849182 y= 298.9698314666748
scara xy (704.8237379682204, 358.9245148536137)
scarxbox (704.8237379682204, 358.9245148536137)
box (704.8237379682204, 358.9245148536137)
scres[i 0.9907348
boxes [0.6815019  0.14610732 0.97366333 0.3811986 ]
rect,width=112.84382343292236height=105.17810583114624
(left, right, top, bottom) (70.131511688

scres[i 0.9877792
boxes [0.68068874 0.19980754 0.9665596  0.4410818 ]
rect,width=115.81164121627808height=102.91350603103638
(left, right, top, bottom) (95.9076189994812, 211.71926021575928, 245.04794597625732, 347.9614520072937)
center= 153.81343960762024 y= 296.5046989917755
scara xy (670.2204768799047, 361.79382417615705)
scarxbox (670.2204768799047, 361.79382417615705)
box (670.2204768799047, 361.79382417615705)
scres[i 0.98488194
boxes [0.68071604 0.2032496  0.96438366 0.44200134]
rect,width=114.60083484649658height=102.12034463882446
(left, right, top, bottom) (97.55980968475342, 212.16064453125, 245.0577735900879, 347.17811822891235)
center= 154.8602271080017 y= 296.1179459095001
scara xy (668.9730714136581, 362.24398830964464)
scarxbox (668.9730714136581, 362.24398830964464)
box (668.9730714136581, 362.24398830964464)
scres[i 0.9968952
boxes [0.68321913 0.20459205 0.96639735 0.44397634]
rect,width=114.90446090698242height=101.9441556930542
(left, right, top, bottom) (98.2041835

In [12]:
import pandas as pd
boxf = pd.DataFrame(boxv)
print(boxf)

             0           1
0   791.660300  359.519077
1   789.763808  355.061668
2   787.963462  356.121716
3   786.987303  357.937595
4   785.486406  360.113376
5   785.272244  359.324203
6   784.573886  359.433747
7   779.822878  364.740096
8   778.573394  364.453886
9   778.183981  357.957401
10  774.554982  364.977417
11  773.300328  359.748679
12  772.131500  357.758331
13  772.620402  360.393255
14  771.014438  359.899371
15  768.152541  360.161742
16  767.770918  358.195933
17  765.355911  358.300332
18  762.206127  362.348550
19  763.418850  357.725201
20  765.072657  358.091059
21  759.411510  362.732291
22  757.163288  356.388146
23  754.485168  356.399172
24  753.820634  361.292959
25  751.966075  361.383022
26  750.127551  361.204420
27  749.567364  361.935524
28  749.021923  362.922868
29  746.649950  354.055342
..         ...         ...
50  713.000016  358.625005
51  710.846859  359.899458
52  708.863100  362.765059
53  710.685691  361.188710
54  707.599091  360.486965
5

In [13]:
boxf.describe()

,0,1
count,80.000000,80.000000
mean,729.872153,360.077347
std,36.921460,2.438325
min,667.267093,354.055342
25%,696.923998,358.543837
50%,730.615720,360.124253
75%,762.509308,361.829249
max,791.660300,366.077552


In [3]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019062600.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.9060332
boxes [0.9385071  0.05851981 0.9960179  0.17671935]
too lower
scres[i 0.91842675
boxes [0.9385663  0.05833279 0.9955292  0.17648658]
too lower
scres[i 0.92191654
boxes [0.9437608  0.05386502 0.99485517 0.18391459]
too lower
scres[i 0.9671182
boxes [0.94338644 0.05509327 0.9950688  0.18290474]
too lower
scres[i 0.92652833
boxes [0.9421168  0.05314352 0.995311   0.18247154]
too lower
scres[i 0.9704232
boxes [0.9376907  0.05812145 0.99569094 0.17464021]
too lower
scres[i 0.9886856
boxes [0.9373783  0.05871989 0.9952919  0.17432995]
too lower
scres[i 0.9985984
boxes [0.7649714  0.0128432  0.96264976 0.24083126]
rect,width=109.4342677295208height=71.16421937942505
(left, right, top, bottom) (6.164735108613968, 115.59900283813477, 275.3896951675415, 346.55391454696655)
center= 60.88186897337437 y= 310.971804857254
scara xy (780.9624843587885, 344.9547284469367)
scarxbox (780.9624843587885, 344.9547284469367)
box (780.9624843587885, 344.9547284469367)
scres[i 0.9990995
boxes

scres[i 0.99224347
boxes [0.7529861  0.02774044 0.9920439  0.23854886]
rect,width=101.18804186582565height=86.06082201004028
(left, right, top, bottom) (13.315410912036896, 114.50345277786255, 271.0749864578247, 357.135808467865)
center= 63.90943184494972 y= 314.10539746284485
scara xy (777.3546857616211, 341.3073599716819)
scarxbox (777.3546857616211, 341.3073599716819)
box (777.3546857616211, 341.3073599716819)
scres[i 0.9878341
boxes [0.7526955  0.02923576 0.9887453  0.23691039]
too lower
scres[i 0.9957749
boxes [0.7622641  0.01045176 0.9667443  0.2387263 ]
rect,width=109.571782797575height=73.6128830909729
(left, right, top, bottom) (5.016842633485794, 114.58862543106079, 274.415066242218, 348.0279493331909)
center= 59.80273403227329 y= 311.22150778770447
scara xy (782.2484366880403, 344.6640848594203)
scarxbox (782.2484366880403, 344.6640848594203)
box (782.2484366880403, 344.6640848594203)
scres[i 0.99387085
boxes [0.7617978  0.00833444 0.9659487  0.23569836]
rect,width=109.13468

scres[i 0.9906019
boxes [0.76216435 0.00820122 0.96783936 0.23767567]
rect,width=110.14773577451706height=74.04300212860107
(left, right, top, bottom) (3.9365842938423157, 114.08432006835938, 274.3791675567627, 348.42216968536377)
center= 59.010452181100845 y= 311.40066862106323
scara xy (783.1925602198941, 344.45554927172867)
scarxbox (783.1925602198941, 344.45554927172867)
box (783.1925602198941, 344.45554927172867)
scres[i 0.98662335
boxes [0.76176757 0.01032668 0.96245843 0.23664753]
rect,width=108.6340083181858height=72.24871158599854
(left, right, top, bottom) (4.9568067491054535, 113.59081506729126, 274.2363238334656, 346.4850354194641)
center= 59.27381090819836 y= 310.36067962646484
scara xy (782.878728505582, 345.6660522154561)
scarxbox (782.878728505582, 345.6660522154561)
box (782.878728505582, 345.6660522154561)
scres[i 0.9932899
boxes [0.76312053 0.01111476 0.9614976  0.23938507]
rect,width=109.5697483420372height=71.41574621200562
(left, right, top, bottom) (5.33508449792

scres[i 0.9984567
boxes [0.7619862  0.01333205 0.96201426 0.23991656]
rect,width=108.76056522130966height=72.01010227203369
(left, right, top, bottom) (6.399385035037994, 115.15995025634766, 274.3150305747986, 346.3251328468323)
center= 60.779667645692825 y= 310.32008171081543
scara xy (781.0842726823605, 345.7133064620472)
scarxbox (781.0842726823605, 345.7133064620472)
box (781.0842726823605, 345.7133064620472)
scres[i 0.9951304
boxes [0.76046556 0.00775953 0.9681701  0.23663522]
rect,width=109.86033126711845height=74.77363586425781
(left, right, top, bottom) (3.724575787782669, 113.58490705490112, 273.76760244369507, 348.5412383079529)
center= 58.654741421341896 y= 311.154420375824
scara xy (783.6164433346279, 344.74217175263766)
scarxbox (783.6164433346279, 344.74217175263766)
box (783.6164433346279, 344.74217175263766)
scres[i 0.9985001
boxes [0.76049954 0.01067182 0.96759564 0.23722064]
rect,width=108.74343797564507height=74.55459594726562
(left, right, top, bottom) (5.1224716007

scres[i 0.9974341
boxes [0.760188   0.01218517 0.9647935  0.23984845]
rect,width=109.27837282419205height=73.65798711776733
(left, right, top, bottom) (5.848883092403412, 115.12725591659546, 273.66767406463623, 347.32566118240356)
center= 60.488069504499435 y= 310.4966676235199
scara xy (781.4317559252722, 345.507767972448)
scarxbox (781.4317559252722, 345.507767972448)
box (781.4317559252722, 345.507767972448)
scres[i 0.99794203
boxes [0.75973535 0.01367598 0.9613332  0.23708549]
rect,width=107.23656311631203height=72.5752329826355
(left, right, top, bottom) (6.5644727647304535, 113.80103588104248, 273.5047245025635, 346.079957485199)
center= 60.18275432288647 y= 309.7923409938812
scara xy (781.795585094357, 346.3275742040057)
scarxbox (781.795585094357, 346.3275742040057)
box (781.795585094357, 346.3275742040057)
scres[i 0.99523574
boxes [0.7528471  0.02947695 0.99056154 0.23258884]
too lower
scres[i 0.9926721
boxes [0.7634702  0.01238737 0.9663254  0.23623304]
rect,width=107.4459215

In [4]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture(1)
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.81533015
boxes [0.7087611  0.03128544 0.9590441  0.21029083]
too lower
scres[i 0.9956275
boxes [0.7078936  0.01497519 0.96892124 0.22538199]
rect,width=100.99526390433311height=93.96994829177856
(left, right, top, bottom) (7.188089042901993, 108.18335294723511, 254.84169960021973, 348.8116478919983)
center= 57.68572099506855 y= 301.826673746109
scara xy (784.7711775822163, 355.59927198375476)
scarxbox (784.7711775822163, 355.59927198375476)
box (784.7711775822163, 355.59927198375476)
scres[i 0.9996644
boxes [0.7277807  0.02129849 1.         0.2262973 ]
too lower
scres[i 0.9979766
boxes [0.7076922  0.01275408 0.96993065 0.22619966]
rect,width=102.45387777686119height=94.40583944320679
(left, right, top, bottom) (6.121957451105118, 108.57583522796631, 254.76919412612915, 349.17503356933594)
center= 57.34889633953571 y= 301.97211384773254
scara xy (785.1725550502556, 355.4299858931656)
scarxbox (785.1725550502556, 355.4299858931656)
box (785.1725550502556, 355.4299858931656)
scr

scres[i 0.998911
boxes [0.72389865 0.01639177 0.985299   0.22043672]
too lower
scres[i 0.99371815
boxes [0.7050665  0.01193563 0.9713605  0.22802517]
rect,width=103.72298061847687height=95.86584091186523
(left, right, top, bottom) (5.7291001081466675, 109.45208072662354, 253.82394075393677, 349.689781665802)
center= 57.5905904173851 y= 301.7568612098694
scara xy (784.8845400388432, 355.6805308057866)
scarxbox (784.8845400388432, 355.6805308057866)
box (784.8845400388432, 355.6805308057866)
scres[i 0.99864715
boxes [0.7077388  0.01101597 0.97098595 0.22575948]
rect,width=103.07688385248184height=94.768967628479
(left, right, top, bottom) (5.287664830684662, 108.3645486831665, 254.78597402572632, 349.5549416542053)
center= 56.82610675692558 y= 302.1704578399658
scara xy (785.7955378264228, 355.19912192468513)
scarxbox (785.7955378264228, 355.19912192468513)
box (785.7955378264228, 355.19912192468513)
scres[i 0.9973416
boxes [0.7219467  0.01444612 0.98500746 0.21889026]
too lower
scres[i 

scres[i 0.9989656
boxes [0.7216709  0.01653093 0.9870034  0.22067319]
too lower
scres[i 0.9994473
boxes [0.72128177 0.02037433 1.         0.22704238]
too lower
scres[i 0.99728346
boxes [0.72293574 0.01626514 0.9858345  0.21985666]
too lower
scres[i 0.9970668
boxes [0.7070052  0.01081068 0.97146916 0.22446887]
rect,width=102.55593225359917height=95.20702600479126
(left, right, top, bottom) (5.189126282930374, 107.74505853652954, 254.52187299728394, 349.7288990020752)
center= 56.46709240972996 y= 302.12538599967957
scara xy (786.2233576647277, 355.2515836292119)
scarxbox (786.2233576647277, 355.2515836292119)
box (786.2233576647277, 355.2515836292119)
scres[i 0.9995378
boxes [0.7254223  0.02057198 1.         0.22550075]
too lower
scres[i 0.9969469
boxes [0.7068347  0.01313799 0.97059643 0.22874159]
rect,width=103.48972678184509height=94.95423316955566
(left, right, top, bottom) (6.30623459815979, 109.79596138000488, 254.46048259735107, 349.41471576690674)
center= 58.051097989082336 y= 30

scres[i 0.99963415
boxes [0.72486836 0.01971861 1.         0.22644128]
too lower
scres[i 0.9977831
boxes [0.7057184  0.01100255 0.97061545 0.22372921]
rect,width=102.108795940876height=95.36293745040894
(left, right, top, bottom) (5.281223952770233, 107.39001989364624, 254.0586233139038, 349.42156076431274)
center= 56.33562192320824 y= 301.7400920391083
scara xy (786.3800246133467, 355.70004940711374)
scarxbox (786.3800246133467, 355.70004940711374)
box (786.3800246133467, 355.70004940711374)
scres[i 0.9996184
boxes [0.7252331  0.01814357 1.         0.2215842 ]
too lower
capture end


In [5]:
import pandas as pd
boxf = pd.DataFrame(boxv)
print(boxf)

             0           1
0   784.771178  355.599272
1   785.172555  355.429986
2   785.777187  354.456491
3   785.159305  356.873151
4   785.874212  355.191130
5   784.860626  355.554865
6   785.063370  355.835706
7   786.758420  350.073685
8   785.266009  354.814007
9   785.502895  355.433707
10  785.205640  355.116839
11  785.139289  355.377924
12  785.585697  353.012951
13  783.026686  344.784156
14  785.057848  355.015525
15  784.828875  355.472507
16  785.040450  354.123651
17  784.884540  355.680531
18  785.795538  355.199122
19  785.123650  355.129926
20  785.203595  355.316596
21  785.772597  354.722546
22  785.274530  354.790480
23  784.801709  356.028007
24  785.122807  355.358905
25  785.801559  356.018254
26  785.851119  354.780490
27  786.188890  356.199129
28  785.865490  355.249573
29  785.267239  354.594695
30  785.204233  354.978660
31  785.528068  354.362170
32  784.919326  355.448893
33  786.223358  355.251584
34  784.335776  355.470159
35  785.480382  356.100424
3

In [6]:
boxf.describe()

,0,1
count,49.000000,49.000000
mean,785.325304,354.562977
std,0.679513,2.327255
min,783.026686,344.784156
25%,785.063370,354.722546
50%,785.274530,355.199122
75%,785.772597,355.554865
max,786.758420,356.873151


In [7]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture(1)
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.8476507
boxes [0.6838502  0.1279209  0.94221836 0.32537884]
too lower
scres[i 0.975973
boxes [0.67252547 0.11579729 0.94725305 0.3363897 ]
rect,width=105.88435292243958height=98.90192985534668
(left, right, top, bottom) (55.582698583602905, 161.46705150604248, 242.10916757583618, 341.01109743118286)
center= 108.52487504482269 y= 291.5601325035095
scara xy (724.1886442238133, 367.5490892035656)
scarxbox (724.1886442238133, 367.5490892035656)
box (724.1886442238133, 367.5490892035656)
scres[i 0.985635
boxes [0.6722583  0.11360251 0.9490002  0.33512336]
rect,width=106.330007314682height=99.62707042694092
(left, right, top, bottom) (54.529205560684204, 160.8592128753662, 242.0129942893982, 341.6400647163391)
center= 107.69420921802521 y= 291.82652950286865
scara xy (725.1785080620745, 367.2390144300831)
scarxbox (725.1785080620745, 367.2390144300831)
box (725.1785080620745, 367.2390144300831)
scres[i 0.9912277
boxes [0.67385024 0.11892194 0.94952244 0.33719152]
rect,width=104.769

scres[i 0.9888515
boxes [0.6713856  0.1136378  0.94904494 0.3324198 ]
rect,width=105.0153636932373height=99.95736837387085
(left, right, top, bottom) (54.546146392822266, 159.56151008605957, 241.69881105422974, 341.6561794281006)
center= 107.05382823944092 y= 291.67749524116516
scara xy (725.9416187534758, 367.412483970093)
scarxbox (725.9416187534758, 367.412483970093)
box (725.9416187534758, 367.412483970093)
scres[i 0.99450994
boxes [0.67113745 0.11363164 0.9476744  0.33168373]
rect,width=104.66499924659729height=99.55329895019531
(left, right, top, bottom) (54.54318881034851, 159.2081880569458, 241.60948276519775, 341.16278171539307)
center= 106.87568843364716 y= 291.3861322402954
scara xy (726.1538992472937, 367.7516181071636)
scarxbox (726.1538992472937, 367.7516181071636)
box (726.1538992472937, 367.7516181071636)
scres[i 0.9924952
boxes [0.6710038  0.11289974 0.9462078  0.33366147]
rect,width=105.9656274318695height=99.07344102859497
(left, right, top, bottom) (54.1918766498565

scres[i 0.99666315
boxes [0.66689324 0.12004141 0.94594455 0.33814   ]
rect,width=104.68732953071594height=100.45846939086914
(left, right, top, bottom) (57.61987566947937, 162.3072052001953, 240.08156776428223, 340.54003715515137)
center= 109.96354043483734 y= 290.3108024597168
scara xy (722.4742570430815, 369.00325622221004)
scarxbox (722.4742570430815, 369.00325622221004)
box (722.4742570430815, 369.00325622221004)
scres[i 0.9870564
boxes [0.67035973 0.11472005 0.94832194 0.3327186 ]
rect,width=104.63931083679199height=100.0663948059082
(left, right, top, bottom) (55.065622329711914, 159.7049331665039, 241.3295030593872, 341.3958978652954)
center= 107.38527774810791 y= 291.3627004623413
scara xy (725.5466465850603, 367.778891699805)
scarxbox (725.5466465850603, 367.778891699805)
box (725.5466465850603, 367.778891699805)
scres[i 0.98988616
boxes [0.67092544 0.11475045 0.94523287 0.3335024 ]
rect,width=105.00094056129456height=98.75067472457886
(left, right, top, bottom) (55.080217123

scres[i 0.9929704
boxes [0.66901493 0.11892051 0.9460368  0.33688965]
rect,width=104.6251916885376height=99.72787857055664
(left, right, top, bottom) (57.08184242248535, 161.70703411102295, 240.84537506103516, 340.5732536315918)
center= 109.39443826675415 y= 290.7093143463135
scara xy (723.1524282622269, 368.5394053407669)
scarxbox (723.1524282622269, 368.5394053407669)
box (723.1524282622269, 368.5394053407669)
scres[i 0.9932221
boxes [0.66944766 0.11928927 0.9481297  0.33571932]
rect,width=103.88642191886902height=100.32553911209106
(left, right, top, bottom) (57.25885033607483, 161.14527225494385, 241.00115776062012, 341.3266968727112)
center= 109.20206129550934 y= 291.16392731666565
scara xy (723.3816744897786, 368.0102551846342)
scarxbox (723.3816744897786, 368.0102551846342)
box (723.3816744897786, 368.0102551846342)
scres[i 0.9954064
boxes [0.67144865 0.11401989 0.9476337  0.3335655 ]
rect,width=105.38189649581909height=99.4266128540039
(left, right, top, bottom) (54.72954511642

scres[i 0.9955498
boxes [0.6677073  0.11957644 0.95148134 0.33864176]
rect,width=105.15135526657104height=102.15864658355713
(left, right, top, bottom) (57.3966908454895, 162.54804611206055, 240.37463665008545, 342.5332832336426)
center= 109.97236847877502 y= 291.453959941864
scara xy (722.4637370948972, 367.6726695482447)
scarxbox (722.4637370948972, 367.6726695482447)
box (722.4637370948972, 367.6726695482447)
scres[i 0.9959313
boxes [0.6716718  0.11618116 0.9489323  0.33230475]
rect,width=103.73932242393494height=99.81377363204956
(left, right, top, bottom) (55.766955614089966, 159.5062780380249, 241.80185079574585, 341.6156244277954)
center= 107.63661682605743 y= 291.70873761177063
scara xy (725.2471380987616, 367.37611917990444)
scarxbox (725.2471380987616, 367.37611917990444)
box (725.2471380987616, 367.37611917990444)
scres[i 0.99160767
boxes [0.67054445 0.11410435 0.94752353 0.3337408 ]
rect,width=105.42549848556519height=99.7124719619751
(left, right, top, bottom) (54.77008581

scres[i 0.9927436
boxes [0.6685552  0.11973471 0.94460124 0.3372923 ]
rect,width=104.4276487827301height=99.37657356262207
(left, right, top, bottom) (57.47266173362732, 161.90031051635742, 240.67987203598022, 340.0564455986023)
center= 109.68648612499237 y= 290.36815881729126
scara xy (722.8044091135083, 368.9364958622005)
scarxbox (722.8044091135083, 368.9364958622005)
box (722.8044091135083, 368.9364958622005)
scres[i 0.9947573
boxes [0.6699958  0.12073588 0.94865865 0.3380022 ]
rect,width=104.28783416748047height=100.3186297416687
(left, right, top, bottom) (57.953224182128906, 162.24105834960938, 241.19848251342773, 341.51711225509644)
center= 110.09714126586914 y= 291.3577973842621
scara xy (722.3150514671041, 367.7845986740894)
scarxbox (722.3150514671041, 367.7845986740894)
box (722.3150514671041, 367.7845986740894)
scres[i 0.9896209
boxes [0.6681735  0.11714306 0.94259524 0.33603084]
rect,width=105.06613612174988height=98.79183053970337
(left, right, top, bottom) (56.228667497

In [8]:
import pandas as pd
boxf = pd.DataFrame(boxv)
print(boxf)

              0           1
0    724.188644  367.549089
1    725.178508  367.239014
2    723.065686  366.796068
3    723.679887  366.210423
4    723.446228  368.016624
5    722.892758  368.115691
6    723.036730  369.430867
7    722.726452  368.927967
8    725.289082  367.758174
9    723.124719  367.816992
10   722.388084  367.834388
11   723.954307  368.593965
12   722.920158  367.791842
13   723.306912  368.578205
14   724.829415  367.505382
15   722.479135  368.737526
16   725.469964  367.840195
17   726.281690  367.408263
18   723.881200  368.011279
19   725.369981  367.406740
20   725.941619  367.412484
21   726.153899  367.751618
22   725.797593  368.086881
23   722.310204  369.132169
24   722.201454  371.436038
25   724.952185  367.462011
26   723.531907  368.379248
27   722.725766  369.021501
28   722.637537  368.951943
29   725.685119  368.171412
..          ...         ...
78   726.292943  367.068430
79   723.015844  368.264322
80   722.463737  367.672670
81   725.247138  367

In [9]:
boxf.describe()

,0,1
count,108.000000,108.000000
mean,724.058127,368.170353
std,1.361789,0.848006
min,721.627172,366.210423
25%,722.827605,367.666291
50%,723.520369,368.049761
75%,725.429400,368.627405
max,726.292943,371.790620


In [10]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture(1)
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.95405674
boxes [0.6730856  0.13124715 0.92809093 0.33059028]
too lower
scres[i 0.8959175
boxes [0.67362267 0.12553844 0.9368028  0.32477066]
too lower
scres[i 0.9991984
boxes [0.6759237  0.12671474 0.9465378  0.33472756]
too lower
scres[i 0.99870765
boxes [0.67419046 0.12531808 0.9436364  0.33247298]
too lower
scres[i 0.98216015
boxes [0.6733603  0.11864957 0.95001894 0.33659226]
rect,width=104.61248874664307height=99.5971155166626
(left, right, top, bottom) (56.95179462432861, 161.56428337097168, 242.40970373153687, 342.00681924819946)
center= 109.25803899765015 y= 292.20826148986816
scara xy (723.3149685999845, 366.7946946378758)
scarxbox (723.3149685999845, 366.7946946378758)
box (723.3149685999845, 366.7946946378758)
scres[i 0.9934227
boxes [0.6711771  0.11342834 0.95181865 0.33371305]
rect,width=105.73666334152222height=101.03096008300781
(left, right, top, bottom) (54.445602893829346, 160.18226623535156, 241.62375211715698, 342.6547122001648)
center= 107.31393456459045 

scres[i 0.9941427
boxes [0.6658838  0.40480667 0.9457442  0.621743  ]
rect,width=104.12944793701172height=100.74975728988647
(left, right, top, bottom) (194.30720329284668, 298.4366512298584, 239.71816062927246, 340.46791791915894)
center= 246.37192726135254 y= 290.0930392742157
scara xy (559.9230541415598, 369.2567233054712)
scarxbox (559.9230541415598, 369.2567233054712)
box (559.9230541415598, 369.2567233054712)
scres[i 0.99512416
boxes [0.6678592  0.44025192 0.9401179  0.6472913 ]
too lower
scres[i 0.9901419
boxes [0.6574646  0.46928772 0.93889195 0.6868672 ]
rect,width=104.43813800811768height=101.31383657455444
(left, right, top, bottom) (225.25810718536377, 329.69624519348145, 236.68726444244385, 338.0011010169983)
center= 277.4771761894226 y= 287.34418272972107
scara xy (522.8564503392615, 372.4562753740775)
scarxbox (522.8564503392615, 372.4562753740775)
box (522.8564503392615, 372.4562753740775)
scres[i 0.9974026
boxes [0.65640384 0.4955366  0.93959635 0.7094515 ]
rect,width=

In [11]:
import pandas as pd
boxf = pd.DataFrame(boxv)
print(boxf)

             0           1
0   723.314969  366.794695
1   725.631663  366.875042
2   725.622596  367.002294
3   723.902485  367.997967
4   723.061554  367.062335
5   725.852083  367.188014
6   724.005075  366.704232
7   722.770297  367.669173
8   725.546056  367.419602
9   726.041048  367.849311
10  704.663640  367.779791
11  692.429513  367.724157
12  671.251227  369.649555
13  644.850500  368.017448
14  625.586598  367.975651
15  611.137656  369.160754
16  591.197511  369.693788
17  574.096307  370.238610
18  559.923054  369.256723
19  522.856450  372.456275
20  508.890340  372.530941
21  491.804862  371.922854
22  478.492322  370.424855
23  463.106102  370.080351
24  443.590761  372.963347
25  417.321140  376.447161
26  403.376815  372.142179
27  359.388746  373.490624


In [12]:
boxf.describe()

,0,1
count,28.000000,28.000000
mean,607.489692,369.447062
std,121.290059,2.521761
min,359.388746,366.704232
25%,504.618971,367.606780
50%,635.218549,368.589101
75%,723.461848,370.799355
max,726.041048,376.447161
